In [1]:
# ------------------------------------------------ #
#                  IMPORT FUNCTIONS                #
# ------------------------------------------------ #
import os
import sys
import datetime

# Get the project root directory (going up one level from Notebooks)
project_root = os.path.dirname(os.getcwd())
# Add the Scripts directory to Python path so we can import modules from individual scripts files
scripts_dir = os.path.join(project_root, 'Scripts')
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Add functions here from Scripts folder
from ao_indices import return_AO_fetch
from enso_indices import return_ENSO_fetch
from breakup_data import calculate_breakup_data
from breakup_data import calculate_estimated_breakup
from bounding_box import calculate_bounding_box
from raster_temp import mean_30d_raster_temps
from raster_temp import monthly_raster_temps
#from raster_temp_era5 import mean_30d_raster_temps
from correlation_function import calculate_ao_correlation_per_month, calculate_enso_correlation_per_month, plot_heatmap
from correlation_raster import calculate_ao_correlation, calculate_enso_correlation, save_results_to_csv, plot_results

In [2]:
# ------------------------------------------------ #
#                CLASS DEFINITION                  #
# ------------------------------------------------ #

class StudySite:
    def __init__(self,name):
        self.name = name
        self.breakup_anomaly_data = None # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_date, breakup_doy
        self.estimated_breakup_doy = None # The MEAN breakup date for the study site across all years, in DOY integer format.
        self.enso_indices = None # Dictionary of integer years (e.g. 2000) with enso_value for each of 12 months before breakup, named as strings
        self.ao_indices = None # Dictionary of integer years (e.g. 2000) with ao_value for each of 12 months before breakup
        self.bounding_box = None  # Dictionary of max_lat:value, min_lat:value, max_lon:value, min_lon:value
        self.mean_30d_raster_temps = None # Dictionary of integer years (e.g. 2000) with rasters of the mean breakup temperature across the 30 days prior to mean breakup date (we can't do actual breakup date because months have different temps)
        self.monthly_raster_temps = None
        self.enso_correlations = None # Single raster of ENSO correlations across all years
        self.ao_correlations = None # Single raster of AO correlations across all years


    def calculate_breakup_data(self, ice_data_filepath, years_of_data):
        """ Load ice breakup data for this site from the provided icedata spreadsheet, filtering by years_of_data and the site name """

        # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_doy
        self.breakup_anomaly_data = calculate_breakup_data(years_of_data, ice_data_filepath, self.name)

        # Single value of estimated breakup day of year for this site (MEAN across all years)
        self.estimated_breakup_doy = calculate_estimated_breakup(self.breakup_anomaly_data)


    def fetch_enso(self,enso_data_filepath):
        """ Pull in ENSO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup 
            date from all years of data (excluding month with breakup date, as we are looking for time lag of 1+ months) """
        
        # Dictionary of integer years (e.g. 2000) with dictionary of preceding 12 month_name: month_values
        self.enso_indices = return_ENSO_fetch(enso_data_filepath, self.breakup_anomaly_data, self.estimated_breakup_doy)


    def fetch_ao(self,ao_data_filepath):
        """ Pull in AO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup 
            date from all years of data (excluding month with breakup date, as we are looking for time lag of 1+ months) """
        
        # Dictionary of integer years (e.g. 2000) with dictionary of preceding 12 month_name: month_values
        self.ao_indices = return_AO_fetch(ao_data_filepath, self.breakup_anomaly_data, self.estimated_breakup_doy)


    def calculate_bounding_box(self, ice_data_filepath):
        """ Calculate 100km bounding box based on study site coordinates """

        # Bounding box for this site based on lat,lon for each site in the ice data CSV
        # Dictionary of max_lat:value, min_lat:value, max_lon:value, min_lon:value
        self.bounding_box = calculate_bounding_box(ice_data_filepath, self.name)


    def calculate_mean_30d_raster_temps(self):
        """ Pull the raster data within the bounding box for each of the 30d prior to the given breakup date variable, then average all the rasters together to return the mean 30d temp for each year."""

        # Dictionary of integer years (e.g. 2000) paired with rasters of the mean breakup temperature across the 30 days prior to given breakup date variable for each year
        # (Note we can't do actual breakup date for each year for each site due to statistical reasons, so for each site we will find the mean breakup date and use that to find our 30d period)
        self.mean_30d_raster_temps = mean_30d_raster_temps(self.bounding_box, self.breakup_anomaly_data, self.estimated_breakup_doy)
       
    def calculate_monthly_raster_temps(self):
        self.monthly_raster_temps_data = monthly_raster_temps(self.bounding_box, self.breakup_anomaly_data, self.estimated_breakup_doy)
        return self.monthly_raster_temps_data
    
    # For each month in our indices in the 12 months prior to breakup (excluding breakup month,) calculate the P value across all years of data
    # Single raster for each of these consisting of correlation across all years
    def calculate_ao_correlation(self, ice_data_filepath, ao_data_filepath, project_root, lag=1):
        """Calculate AO correlations for this study site."""
        self.ao_correlations = calculate_ao_correlation(
            site_name=self.name,
            ice_data_filepath=ice_data_filepath,
            ao_data_filepath=ao_data_filepath,
            project_root=project_root,
            lag=lag
        )

    def calculate_enso_correlation(self, ice_data_filepath, enso_data_filepath, project_root, lag=1):
        """Calculate ENSO correlations for this study site."""
        self.enso_correlations = calculate_enso_correlation(
            site_name=self.name,
            ice_data_filepath=ice_data_filepath,
            enso_data_filepath=enso_data_filepath,
            project_root=project_root,
            lag=lag
        )

In [3]:
# ------------------------------------------------ #
#                  PARAMETERS                      #
# ------------------------------------------------ #

# Add data here from Data folder
enso_data_filepath = os.path.join(project_root, "Data", "ENSO_index.txt")
ao_data_filepath = os.path.join(project_root, "Data", "AO_index.txt")
ice_data_filepath = os.path.join(project_root, "Data", "ice_data.csv")
results_dir = os.path.join(project_root, "Results")
os.makedirs(results_dir, exist_ok=True)

# Choose study sites here "Gambell","Teller","Nuiqsut","Shishmaref","Savoonga","Prudhoe Bay","Kaktovik" miss some data
study_site_names = ["Nunam Iqua","Stebbins","Hooper Bay","Mekoryuk","Utqiagvik","Kotzebue","Elim","Toksook Bay","Shaktoolik","Kwigillingok","Scammon Bay","Nome","Point Hope","Chefornak","Tununak","Kotlik","Newtok","Brevig Mission","Wainwright","Koyuk","Alakanuk","Quinhagak","Kivalina","Emmonak","Golovin","Kipnuk","Unalakleet","Point Lay","Deering","Wales","Kongiganak"]

# Generate a list of years from 2000 to 2022 - select years
years_of_data = list(range(2000, 2023))

In [4]:
# ------------------------------------------------ #
#                  MAIN CODE                      #
# ------------------------------------------------ #

# Create list of StudySite objects
sites = [StudySite(name) for name in study_site_names]

# Process data for each site
for site in sites:
    # Calculate basic site data
    site.calculate_breakup_data(ice_data_filepath, years_of_data)
    
    site.fetch_enso(enso_data_filepath)
        
    site.fetch_ao(ao_data_filepath)
      
    site.calculate_bounding_box(ice_data_filepath)
    
    site.calculate_mean_30d_raster_temps()
    
    site.calculate_monthly_raster_temps()
    
    site.calculate_ao_correlation(
        ice_data_filepath=ice_data_filepath,
        ao_data_filepath=ao_data_filepath,
        project_root=project_root,
        lag=1
    )
    site.calculate_enso_correlation(
        ice_data_filepath=ice_data_filepath,
        enso_data_filepath=enso_data_filepath,
        project_root=project_root,
        lag=1
    )
    
    #print(f"Finished processing site: {site.name}")
    #print(site.monthly_raster_temps_data)
    #print(site.breakup_anomaly_data) # Dictionary of integer years (e.g. 2000) with enso_value for each of 12 months before breakup
    """ self.enso_correlations = None # Single raster of ENSO correlations across all years
        self.ao_correlations = None # Single raster of AO correlations across all years"""

Results saved to /home/jovyan/work/ESDA_project/cee506_class_project/Results/ao_correlations_Nunam Iqua.csv
AO correlations saved for Nunam Iqua
Results saved to /home/jovyan/work/ESDA_project/cee506_class_project/Results/enso_correlations_Nunam Iqua.csv
ENSO correlations saved for Nunam Iqua
Results saved to /home/jovyan/work/ESDA_project/cee506_class_project/Results/ao_correlations_Stebbins.csv
AO correlations saved for Stebbins
Results saved to /home/jovyan/work/ESDA_project/cee506_class_project/Results/enso_correlations_Stebbins.csv
ENSO correlations saved for Stebbins
Results saved to /home/jovyan/work/ESDA_project/cee506_class_project/Results/ao_correlations_Hooper Bay.csv
AO correlations saved for Hooper Bay
Results saved to /home/jovyan/work/ESDA_project/cee506_class_project/Results/enso_correlations_Hooper Bay.csv
ENSO correlations saved for Hooper Bay
Results saved to /home/jovyan/work/ESDA_project/cee506_class_project/Results/ao_correlations_Mekoryuk.csv
AO correlations save

In [7]:
for site in sites:
    if site.ao_correlations is not None:
        if site.enso_correlations is not None:
            monthly_temp = site.monthly_raster_temps_data
            #print(monthly_temp)
            ao_indices = site.ao_indices
            enso_indices = site.enso_indices
            corr_matrix = plot_heatmap(monthly_temp, ao_indices, enso_indices)
            print(corr_matrix)

          Avg_temp        AO      ENSO
Avg_temp  1.000000 -0.072666  0.233112
AO       -0.072666  1.000000  0.006068
ENSO      0.233112  0.006068  1.000000
          Avg_temp        AO      ENSO
Avg_temp  1.000000 -0.073174  0.257911
AO       -0.073174  1.000000  0.006068
ENSO      0.257911  0.006068  1.000000
          Avg_temp        AO      ENSO
Avg_temp  1.000000 -0.069597  0.220507
AO       -0.069597  1.000000  0.006068
ENSO      0.220507  0.006068  1.000000
          Avg_temp        AO      ENSO
Avg_temp  1.000000 -0.074713  0.200840
AO       -0.074713  1.000000  0.006068
ENSO      0.200840  0.006068  1.000000
          Avg_temp        AO      ENSO
Avg_temp  1.000000 -0.063154  0.225470
AO       -0.063154  1.000000 -0.020321
ENSO      0.225470 -0.020321  1.000000
          Avg_temp        AO      ENSO
Avg_temp  1.000000 -0.077815  0.267075
AO       -0.077815  1.000000 -0.000846
ENSO      0.267075 -0.000846  1.000000
          Avg_temp        AO      ENSO
Avg_temp  1.000000 -0.074

In [6]:
enso_indices = site.enso_indices
print(enso_indices)

{2001: [{'month_name': 'May', 'month_value': 0.8}, {'month_name': 'June', 'month_value': 1.2}, {'month_name': 'July', 'month_value': 0.3}, {'month_name': 'August', 'month_value': -0.4}, {'month_name': 'September', 'month_value': 0.1}, {'month_name': 'October', 'month_value': -1.5}, {'month_name': 'November', 'month_value': -2.2}, {'month_name': 'December', 'month_value': -2.4}, {'month_name': 'January', 'month_value': 1.0}, {'month_name': 'February', 'month_value': -2.9}, {'month_name': 'March', 'month_value': 0.5}, {'month_name': 'April', 'month_value': 0.0}, {'month_name': 'May', 'month_value': 1.3}], 2002: [{'month_name': 'May', 'month_value': 1.3}, {'month_name': 'June', 'month_value': -0.1}, {'month_name': 'July', 'month_value': 0.6}, {'month_name': 'August', 'month_value': 0.7}, {'month_name': 'September', 'month_value': -0.1}, {'month_name': 'October', 'month_value': -0.7}, {'month_name': 'November', 'month_value': -0.3}, {'month_name': 'December', 'month_value': 0.4}, {'month_n

In [ ]:
# ------------------------------------------------ #
#       CORRELATION AND PLOTTING (Optional)        #
# ------------------------------------------------ #

# Save and plot AO and ENSO correlations for each site
for site in sites:
    if site.ao_correlations is not None:
        # Save AO correlations to CSV
        ao_csv_path = os.path.join(results_dir, f"ao_correlation_{site.name}.csv")
        save_results_to_csv(site.ao_correlations, ao_csv_path)
        
        # Plot AO correlations
        ao_plot_dir = os.path.join(results_dir, "AO_Plots")
        plot_results(site.ao_correlations, "AO", ao_plot_dir)
    
    if site.enso_correlations is not None:
        # Save ENSO correlations to CSV
        enso_csv_path = os.path.join(results_dir, f"enso_correlation_{site.name}.csv")
        save_results_to_csv(site.enso_correlations, enso_csv_path)
        
        # Plot ENSO correlations
        enso_plot_dir = os.path.join(results_dir, "ENSO_Plots")
        plot_results(site.enso_correlations, "ENSO", enso_plot_dir)